# import 및 drive 연결

In [ ]:
# Google Drive와 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# 모델
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 한글 호환 문제
from matplotlib import font_manager as fm
from matplotlib import rc

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rcParams['font.family'] = 'NanumBarunGothic'
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rcParams['font.family'] = font_name

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 33 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 3s (3,568 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121749 files and dire

# 데이터 불러오기

In [ ]:
research = pd.read_csv('/content/drive/MyDrive/3조(Rodi알려조)/9. 데이터셋/MBTI설문조사_ver1.csv')
traveler = pd.read_csv('/content/drive/MyDrive/3조(Rodi알려조)/9. 데이터셋/001.권역별데이터(원본)/Training/여행객.csv')
codeB = pd.read_csv('/content/drive/MyDrive/3조(Rodi알려조)/9. 데이터셋/000.국내 여행로그 데이터(동부권)/01-1.정식개방데이터/Training/01.원천데이터/TS_csv/tc_codeb_코드B.csv')

### 설문조사

In [ ]:
research.columns

Index(['타임스탬프', '성별', '연령', '거주지', 'MBTI', '여행 선호도 [자연vs도시]',
       '여행 선호도 [숙박vs당일]', '여행 선호도 [익숙한지역(가본곳)vs새로운지역(안가본곳)]',
       '여행 선호도 [편하지만비싼숙소vs불편하지만저렴한숙소]', '여행 선호도 [휴양/휴식vs체험/활동]',
       '여행 선호도 [잘알려지지않은vs알려진]', '여행 선호도 [계획에 따른 vs 상황에 따른]',
       '여행 선호도 [사진촬영 중요하지않음vs사진촬영 중요]', '여행 동기 [1순위]', '여행 동기 [2순위]',
       '여행 동기 [3순위]', '좋았던 국내 여행지 1순위 시도', '좋았던  국내  여행지 1순위 세부',
       '좋았던 국내 여행지 2순위 시도', '좋았던 국내  여행지 2순위 세부', '좋았던 국내 여행지 3순위 시도',
       '좋았던 국내 여행지 3순위 시도.1', '이메일 주소'],
      dtype='object')

In [ ]:
research.columns = ['타임스탬프', '성별', '연령', '거주지', 'MBTI', '자연/도시',
                    '숙박/당일', '익숙한/새로운',
                    '편하지만비싼숙소/불편하지만저렴한숙소', '휴양휴식/체험활동',
                    '잘알려지지않은/알려진', '계획에따른/상황에따른',
                    '사진촬영중요하지않음/사진촬영중요', '여행동기1순위', '여행동기2순위', '여행동기3순위',
                    '좋았던국내여행지 1순위 시도', '좋았던국내여행지 1순위 세부', '좋았던국내여행지 2순위 시도',
                    '좋았던국내여행지 2순위 세부', '좋았던국내여행지 3순위 시도', '좋았던국내여행지 3순위 세부', '이메일 주소']

In [ ]:
research.head()

,타임스탬프,성별,연령,거주지,MBTI,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,...,여행동기1순위,여행동기2순위,여행동기3순위,좋았던국내여행지 1순위 시도,좋았던국내여행지 1순위 세부,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부,이메일 주소
0,2024. 2. 7 오후 4:44:43,여자,20대,울산광역시,ENFP,후자 조금선호,전자 조금선호,후자 조금선호,전자 중간선호,후자 중간선호,...,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",새로운 경험 추구,여행 동반자와의 친밀감 및 유대감 증진,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,NaN
1,2024. 2. 7 오후 4:45:59,여자,30대,충청남도,INFP,전자 중간선호,전자 조금선호,후자 매우선호,전자 중간선호,전자 조금선호,...,여행 동반자와의 친밀감 및 유대감 증진,진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,새로운 경험 추구,강원도,강릉시,전라남도,여수시,경상남도,통영시,NaN
2,2024. 2. 7 오후 4:48:04,여자,20대,경기도,INTJ,전자 중간선호,전자 중간선호,후자 중간선호,전자 조금선호,후자 중간선호,...,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식","일상적인 환경 및 역할에서의 탈출, 지루함 탈피",새로운 경험 추구,강원도,강릉시,제주특별자치도,서귀포시,부산광역시,부산광역시,NaN
3,2024. 2. 7 오후 4:48:08,여자,20대,울산광역시,ENFP,후자 조금선호,전자 조금선호,후자 조금선호,전자 중간선호,후자 중간선호,...,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",새로운 경험 추구,여행 동반자와의 친밀감 및 유대감 증진,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,NaN
4,2024. 2. 7 오후 4:49:06,남자,20대,경기도,ENTP,전자 중간선호,전자 중간선호,후자 중간선호,전자 중간선호,전자 중간선호,...,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피","쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",새로운 경험 추구,제주특별자치도,서귀포시,강원도,양양군,NaN,NaN,NaN


In [ ]:
research_label = research.copy()
research_oh = research.copy()

### AI HUB

In [ ]:
traveler.columns

Index(['TRAVELER_ID', 'RESIDENCE_SGG_CD', 'GENDER', 'AGE_GRP', 'EDU_NM',
       'EDU_FNSH_SE', 'MARR_STTS', 'FAMILY_MEMB', 'JOB_NM', 'JOB_ETC',
       'INCOME', 'HOUSE_INCOME', 'TRAVEL_TERM', 'TRAVEL_NUM',
       'TRAVEL_LIKE_SIDO_1', 'TRAVEL_LIKE_SGG_1', 'TRAVEL_LIKE_SIDO_2',
       'TRAVEL_LIKE_SGG_2', 'TRAVEL_LIKE_SIDO_3', 'TRAVEL_LIKE_SGG_3',
       'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
       'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
       'TRAVEL_STATUS_RESIDENCE', 'TRAVEL_STATUS_DESTINATION',
       'TRAVEL_STATUS_ACCOMPANY', 'TRAVEL_STATUS_YMD', 'TRAVEL_MOTIVE_1',
       'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3', 'TRAVEL_COMPANIONS_NUM'],
      dtype='object')

In [ ]:
# AI HUB 여행스타일 중립 삭제
columns = ['TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
           'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8']
for column in columns:
  traveler.drop(traveler[traveler[column] == 4].index, inplace=True)
traveler.reset_index(inplace=True, drop=True)
traveler

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,...,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,a006685,41,여,30,5,1.0,2.0,2,3.0,NaN,...,5,2,경기도,인천,2인 가족 여행,2022-10-09~2022-10-09,10.0,NaN,NaN,1
1,a014274,11,여,20,6,1.0,1.0,4,2.0,NaN,...,2,6,서울특별시,경기,나홀로 여행,2022-11-01~2022-11-03,4.0,7.0,1.0,0
2,a001158,41,남,30,6,1.0,1.0,1,3.0,NaN,...,3,2,경기도,경기,나홀로 여행,2022-09-01~2022-09-03,8.0,1.0,6.0,0
3,a005594,28,남,30,6,1.0,2.0,4,3.0,NaN,...,6,2,인천광역시,경기,자녀 동반 여행,2022-10-08~2022-10-08,3.0,1.0,2.0,3
4,a007033,11,남,20,4,1.0,1.0,3,12.0,NaN,...,7,7,서울특별시,경기,나홀로 여행,2022-10-15~2022-10-15,1.0,2.0,7.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3799,d000922,41,남,50,6,1.0,1.0,2,3.0,NaN,...,3,3,경기도,제주,부모 동반 여행,2022-08-23~2022-08-26,1.0,3.0,2.0,4
3800,d010626,50,여,40,5,1.0,2.0,4,13.0,NaN,...,5,2,제주특별자치도,제주,자녀 동반 여행,2022-11-05~2022-11-06,1.0,2.0,3.0,3
3801,d001143,11,여,30,7,1.0,2.0,4,2.0,NaN,...,5,5,서울특별시,제주,자녀 동반 여행,2022-09-17~2022-09-21,1.0,7.0,3.0,3
3802,b004853,48,여,30,7,1.0,1.0,3,3.0,NaN,...,2,6,경상남도,부산,나홀로 여행,2022-09-30~2022-10-01,2.0,1.0,5.0,0


In [ ]:
traveler_rev = traveler[['GENDER', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
                     'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
                     'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3']]

traveler_rev.columns = ['성별', '자연/도시', '숙박/당일', '익숙한/새로운', '편하지만비싼숙소/불편하지만저렴한숙소',
                    '휴양휴식/체험활동', '잘알려지지않은/알려진', '계획에따른/상황에따른', '사진촬영중요하지않음/사진촬영중요',
                    '여행동기1순위', '여행동기2순위', '여행동기3순위']

In [ ]:
traveler_label = traveler_rev.copy()
traveler_oh = traveler_rev.copy()

In [ ]:
# AI HUB 원핫인코딩할 df 역라벨링
def merge_codeB(df, code_a, merge_on_col):
    codeB_rev = codeB.loc[codeB['cd_a'] == code_a][['cd_nm', 'cd_b']]
    codeB_rev['cd_b'] = codeB_rev['cd_b'].astype(int)
    df = pd.merge(codeB_rev, df, left_on='cd_b', right_on=merge_on_col, how='right')
    df.drop(columns=['cd_b', merge_on_col], inplace=True)
    df.rename(columns={'cd_nm': merge_on_col}, inplace=True)
    return df

# 여행동기 수정
traveler_oh = merge_codeB(traveler_oh, 'TMT', '여행동기1순위')
traveler_oh = merge_codeB(traveler_oh, 'TMT', '여행동기2순위')
traveler_oh = merge_codeB(traveler_oh, 'TMT', '여행동기3순위')

def merge_travelstyle(df, style_col):
    df.loc[df[style_col]==1, style_col] = '전자 매우선호'
    df.loc[df[style_col]==2, style_col] = '전자 중간선호'
    df.loc[df[style_col]==3, style_col] = '전자 조금선호'
    df.loc[df[style_col]==5, style_col] = '후자 조금선호'
    df.loc[df[style_col]==6, style_col] = '후자 중간선호'
    df.loc[df[style_col]==7, style_col] = '후자 매우선호'
    return df

# 여행스타일 수정
traveler_oh = merge_travelstyle(traveler_oh, '자연/도시')
traveler_oh = merge_travelstyle(traveler_oh, '숙박/당일')
traveler_oh = merge_travelstyle(traveler_oh, '익숙한/새로운')
traveler_oh = merge_travelstyle(traveler_oh, '편하지만비싼숙소/불편하지만저렴한숙소')
traveler_oh = merge_travelstyle(traveler_oh, '휴양휴식/체험활동')
traveler_oh = merge_travelstyle(traveler_oh, '잘알려지지않은/알려진')
traveler_oh = merge_travelstyle(traveler_oh, '계획에따른/상황에따른')
traveler_oh = merge_travelstyle(traveler_oh, '사진촬영중요하지않음/사진촬영중요')

traveler_oh.head()

,여행동기3순위,여행동기2순위,여행동기1순위,성별,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,계획에따른/상황에따른,사진촬영중요하지않음/사진촬영중요
0,NaN,NaN,기타,여,전자 매우선호,전자 매우선호,후자 매우선호,후자 조금선호,전자 매우선호,전자 매우선호,후자 조금선호,전자 중간선호
1,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",새로운 경험 추구,진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,여,전자 중간선호,전자 매우선호,전자 매우선호,후자 조금선호,전자 조금선호,전자 조금선호,전자 중간선호,후자 중간선호
2,"운동, 건강 증진 및 충전","일상적인 환경 및 역할에서의 탈출, 지루함 탈피","역사 탐방, 문화적 경험 등 교육적 동기",남,전자 매우선호,전자 조금선호,전자 중간선호,전자 조금선호,전자 중간선호,전자 중간선호,전자 조금선호,전자 중간선호
3,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식","일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행 동반자와의 친밀감 및 유대감 증진,남,전자 중간선호,후자 매우선호,전자 중간선호,후자 중간선호,후자 중간선호,후자 중간선호,후자 중간선호,전자 중간선호
4,새로운 경험 추구,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식","일상적인 환경 및 역할에서의 탈출, 지루함 탈피",남,후자 매우선호,후자 매우선호,전자 매우선호,전자 매우선호,전자 조금선호,전자 매우선호,후자 매우선호,후자 매우선호


# 데이터 전처리

## 여행스타일, 동기, 성별 라벨링

### 설문조사

In [ ]:
# 여행스타일
style_mapping = {
    '전자 매우선호': 1,
    '전자 중간선호': 2,
    '전자 조금선호': 3,
    '중립': 4,
    '후자 조금선호': 5,
    '후자 중간선호': 6,
    '후자 매우선호': 7,
}
style_columns = ['자연/도시',
       '숙박/당일', '익숙한/새로운',
       '편하지만비싼숙소/불편하지만저렴한숙소', '휴양휴식/체험활동',
       '잘알려지지않은/알려진', '계획에따른/상황에따른',
       '사진촬영중요하지않음/사진촬영중요']

for column in style_columns:
    research_label[column] = research_label[column].map(style_mapping)

In [ ]:
# 여행동기
mtv_mapping = {'여행 동반자와의 친밀감 및 유대감 증진': 3,
           '새로운 경험 추구': 7,
           '쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식': 2,
           '진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기': 4,
           '일상적인 환경 및 역할에서의 탈출, 지루함 탈피': 1,
           'SNS 사진 등록 등 과시': 5,
           '운동, 건강 증진 및 충전': 6,
           '역사 탐방, 문화적 경험 등 교육적 동기': 8,
           }
mtv_columns = ['여행동기1순위', '여행동기2순위', '여행동기3순위']

for column in mtv_columns:
    research_label[column] = research_label[column].map(mtv_mapping)

In [ ]:
# 성별
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
research_label['성별'] = le.fit_transform(research_label['성별'])

In [ ]:
research_label.head()

,타임스탬프,성별,연령,거주지,MBTI,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,...,여행동기1순위,여행동기2순위,여행동기3순위,좋았던국내여행지 1순위 시도,좋았던국내여행지 1순위 세부,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부,이메일 주소
0,2024. 2. 7 오후 4:44:43,1,20대,울산광역시,ENFP,5,3,5,2,6,...,1,7,3,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,NaN
1,2024. 2. 7 오후 4:45:59,1,30대,충청남도,INFP,2,3,7,2,3,...,3,4,7,강원도,강릉시,전라남도,여수시,경상남도,통영시,NaN
2,2024. 2. 7 오후 4:48:04,1,20대,경기도,INTJ,2,2,6,3,6,...,2,1,7,강원도,강릉시,제주특별자치도,서귀포시,부산광역시,부산광역시,NaN
3,2024. 2. 7 오후 4:48:08,1,20대,울산광역시,ENFP,5,3,5,2,6,...,1,7,3,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,NaN
4,2024. 2. 7 오후 4:49:06,0,20대,경기도,ENTP,2,2,6,2,2,...,1,2,7,제주특별자치도,서귀포시,강원도,양양군,NaN,NaN,NaN


### AI HUB

In [ ]:
# 성별
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
traveler_label['성별'] = le.fit_transform(traveler_label['성별'])

## 여행스타일, 동기, 성별 원핫인코딩

### 설문조사

In [ ]:
research_oh = pd.get_dummies(research_oh, columns=['성별', '자연/도시', '숙박/당일', '익숙한/새로운', '편하지만비싼숙소/불편하지만저렴한숙소',
                                    '휴양휴식/체험활동', '잘알려지지않은/알려진', '계획에따른/상황에따른', '사진촬영중요하지않음/사진촬영중요',
                                    '여행동기1순위', '여행동기2순위', '여행동기3순위',])
research_oh.head()

,타임스탬프,연령,거주지,MBTI,좋았던국내여행지 1순위 시도,좋았던국내여행지 1순위 세부,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부,...,"여행동기2순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행동기2순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,여행동기3순위_SNS 사진 등록 등 과시,여행동기3순위_새로운 경험 추구,"여행동기3순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행동기3순위_여행 동반자와의 친밀감 및 유대감 증진,"여행동기3순위_역사 탐방, 문화적 경험 등 교육적 동기","여행동기3순위_운동, 건강 증진 및 충전","여행동기3순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행동기3순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기
0,2024. 2. 7 오후 4:44:43,20대,울산광역시,ENFP,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,...,0,0,0,0,0,1,0,0,0,0
1,2024. 2. 7 오후 4:45:59,30대,충청남도,INFP,강원도,강릉시,전라남도,여수시,경상남도,통영시,...,0,1,0,1,0,0,0,0,0,0
2,2024. 2. 7 오후 4:48:04,20대,경기도,INTJ,강원도,강릉시,제주특별자치도,서귀포시,부산광역시,부산광역시,...,1,0,0,1,0,0,0,0,0,0
3,2024. 2. 7 오후 4:48:08,20대,울산광역시,ENFP,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,...,0,0,0,0,0,1,0,0,0,0
4,2024. 2. 7 오후 4:49:06,20대,경기도,ENTP,제주특별자치도,서귀포시,강원도,양양군,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0


### AI HUB

In [ ]:
traveler_oh = pd.get_dummies(traveler_oh, columns=['성별', '자연/도시', '숙박/당일', '익숙한/새로운', '편하지만비싼숙소/불편하지만저렴한숙소',
                                    '휴양휴식/체험활동', '잘알려지지않은/알려진', '계획에따른/상황에따른', '사진촬영중요하지않음/사진촬영중요',
                                    '여행동기1순위', '여행동기2순위', '여행동기3순위',])
traveler_oh.head()

,성별_남,성별_여,자연/도시_전자 매우선호,자연/도시_전자 조금선호,자연/도시_전자 중간선호,자연/도시_후자 매우선호,자연/도시_후자 조금선호,자연/도시_후자 중간선호,숙박/당일_전자 매우선호,숙박/당일_전자 조금선호,...,여행동기3순위_SNS 사진 등록 등 과시,여행동기3순위_기타,여행동기3순위_새로운 경험 추구,"여행동기3순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행동기3순위_여행 동반자와의 친밀감 및 유대감 증진,"여행동기3순위_역사 탐방, 문화적 경험 등 교육적 동기","여행동기3순위_운동, 건강 증진 및 충전","여행동기3순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행동기3순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,"여행동기3순위_특별한 목적(칠순여행, 신혼여행, 수학여행, 인센티브여행)"
0,0,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


## MBTI 요소별 column 추가

### 설문조사

In [ ]:
research_label['EI'] = research_label['MBTI'].str[0]
research_label['SN'] = research_label['MBTI'].str[1]
research_label['FT'] = research_label['MBTI'].str[2]
research_label['JP'] = research_label['MBTI'].str[3]

research_label.head()

,타임스탬프,성별,연령,거주지,MBTI,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,...,좋았던국내여행지 1순위 세부,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부,이메일 주소,EI,SN,FT,JP
0,2024. 2. 7 오후 4:44:43,1,20대,울산광역시,ENFP,5,3,5,2,6,...,부산광역시,서울특별시,서울특별시,경상북도,경주시,NaN,E,N,F,P
1,2024. 2. 7 오후 4:45:59,1,30대,충청남도,INFP,2,3,7,2,3,...,강릉시,전라남도,여수시,경상남도,통영시,NaN,I,N,F,P
2,2024. 2. 7 오후 4:48:04,1,20대,경기도,INTJ,2,2,6,3,6,...,강릉시,제주특별자치도,서귀포시,부산광역시,부산광역시,NaN,I,N,T,J
3,2024. 2. 7 오후 4:48:08,1,20대,울산광역시,ENFP,5,3,5,2,6,...,부산광역시,서울특별시,서울특별시,경상북도,경주시,NaN,E,N,F,P
4,2024. 2. 7 오후 4:49:06,0,20대,경기도,ENTP,2,2,6,2,2,...,서귀포시,강원도,양양군,NaN,NaN,NaN,E,N,T,P


In [ ]:
research_oh['EI'] = research_oh['MBTI'].str[0]
research_oh['SN'] = research_oh['MBTI'].str[1]
research_oh['FT'] = research_oh['MBTI'].str[2]
research_oh['JP'] = research_oh['MBTI'].str[3]

research_oh.head()

,타임스탬프,연령,거주지,MBTI,좋았던국내여행지 1순위 시도,좋았던국내여행지 1순위 세부,좋았던국내여행지 2순위 시도,좋았던국내여행지 2순위 세부,좋았던국내여행지 3순위 시도,좋았던국내여행지 3순위 세부,...,"여행동기3순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행동기3순위_여행 동반자와의 친밀감 및 유대감 증진,"여행동기3순위_역사 탐방, 문화적 경험 등 교육적 동기","여행동기3순위_운동, 건강 증진 및 충전","여행동기3순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행동기3순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,EI,SN,FT,JP
0,2024. 2. 7 오후 4:44:43,20대,울산광역시,ENFP,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,...,0,1,0,0,0,0,E,N,F,P
1,2024. 2. 7 오후 4:45:59,30대,충청남도,INFP,강원도,강릉시,전라남도,여수시,경상남도,통영시,...,0,0,0,0,0,0,I,N,F,P
2,2024. 2. 7 오후 4:48:04,20대,경기도,INTJ,강원도,강릉시,제주특별자치도,서귀포시,부산광역시,부산광역시,...,0,0,0,0,0,0,I,N,T,J
3,2024. 2. 7 오후 4:48:08,20대,울산광역시,ENFP,부산광역시,부산광역시,서울특별시,서울특별시,경상북도,경주시,...,0,1,0,0,0,0,E,N,F,P
4,2024. 2. 7 오후 4:49:06,20대,경기도,ENTP,제주특별자치도,서귀포시,강원도,양양군,NaN,NaN,...,0,0,0,0,0,0,E,N,T,P


## 여행객 성별O/X, 동기O/X로 분리(정확도 기준)

### 설문조사

In [ ]:
# 성별 X, 동기 X, 원핫인코딩
mbti = research_oh[['MBTI', '자연/도시_전자 매우선호',
       '자연/도시_전자 조금선호', '자연/도시_전자 중간선호', '자연/도시_후자 매우선호', '자연/도시_후자 조금선호',
       '자연/도시_후자 중간선호', '숙박/당일_전자 매우선호', '숙박/당일_전자 조금선호', '숙박/당일_전자 중간선호',
       '숙박/당일_후자 매우선호', '숙박/당일_후자 조금선호', '숙박/당일_후자 중간선호', '익숙한/새로운_전자 매우선호',
       '익숙한/새로운_전자 조금선호', '익숙한/새로운_전자 중간선호', '익숙한/새로운_후자 매우선호',
       '익숙한/새로운_후자 조금선호', '익숙한/새로운_후자 중간선호', '편하지만비싼숙소/불편하지만저렴한숙소_전자 매우선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_전자 조금선호', '편하지만비싼숙소/불편하지만저렴한숙소_전자 중간선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_후자 매우선호', '편하지만비싼숙소/불편하지만저렴한숙소_후자 조금선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_후자 중간선호', '휴양휴식/체험활동_전자 매우선호', '휴양휴식/체험활동_전자 조금선호',
       '휴양휴식/체험활동_전자 중간선호', '휴양휴식/체험활동_후자 매우선호', '휴양휴식/체험활동_후자 조금선호',
       '휴양휴식/체험활동_후자 중간선호', '잘알려지지않은/알려진_전자 매우선호', '잘알려지지않은/알려진_전자 조금선호',
       '잘알려지지않은/알려진_전자 중간선호', '잘알려지지않은/알려진_후자 매우선호', '잘알려지지않은/알려진_후자 조금선호',
       '잘알려지지않은/알려진_후자 중간선호', '계획에따른/상황에따른_전자 매우선호', '계획에따른/상황에따른_전자 조금선호',
       '계획에따른/상황에따른_전자 중간선호', '계획에따른/상황에따른_후자 매우선호', '계획에따른/상황에따른_후자 조금선호',
       '계획에따른/상황에따른_후자 중간선호', '사진촬영중요하지않음/사진촬영중요_전자 매우선호',
       '사진촬영중요하지않음/사진촬영중요_전자 조금선호', '사진촬영중요하지않음/사진촬영중요_전자 중간선호',
       '사진촬영중요하지않음/사진촬영중요_후자 매우선호', '사진촬영중요하지않음/사진촬영중요_후자 조금선호',
       '사진촬영중요하지않음/사진촬영중요_후자 중간선호',
       'EI', 'SN', 'FT', 'JP']]
mbti.head()

,MBTI,자연/도시_전자 매우선호,자연/도시_전자 조금선호,자연/도시_전자 중간선호,자연/도시_후자 매우선호,자연/도시_후자 조금선호,자연/도시_후자 중간선호,숙박/당일_전자 매우선호,숙박/당일_전자 조금선호,숙박/당일_전자 중간선호,...,사진촬영중요하지않음/사진촬영중요_전자 매우선호,사진촬영중요하지않음/사진촬영중요_전자 조금선호,사진촬영중요하지않음/사진촬영중요_전자 중간선호,사진촬영중요하지않음/사진촬영중요_후자 매우선호,사진촬영중요하지않음/사진촬영중요_후자 조금선호,사진촬영중요하지않음/사진촬영중요_후자 중간선호,EI,SN,FT,JP
0,ENFP,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,E,N,F,P
1,INFP,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,I,N,F,P
2,INTJ,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,I,N,T,J
3,ENFP,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,E,N,F,P
4,ENTP,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,E,N,T,P


In [ ]:
# 성별 O, 동기 X, 라벨링
mbti_gender = research_label[['MBTI', '성별', '자연/도시', '숙박/당일', '익숙한/새로운', '편하지만비싼숙소/불편하지만저렴한숙소',
                        '휴양휴식/체험활동', '잘알려지지않은/알려진', '계획에따른/상황에따른', '사진촬영중요하지않음/사진촬영중요',
                        'EI', 'SN', 'FT', 'JP']]
mbti_gender.head()

,MBTI,성별,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,계획에따른/상황에따른,사진촬영중요하지않음/사진촬영중요,EI,SN,FT,JP
0,ENFP,1,5,3,5,2,6,5,3,7,E,N,F,P
1,INFP,1,2,3,7,2,3,1,2,2,I,N,F,P
2,INTJ,1,2,2,6,3,6,5,6,6,I,N,T,J
3,ENFP,1,5,3,5,2,6,5,3,7,E,N,F,P
4,ENTP,0,2,2,6,2,2,6,6,5,E,N,T,P


In [ ]:
# 성별 X, 동기 O, 원핫인코딩
mbti_motivation = research_oh[['MBTI', '여행동기1순위_새로운 경험 추구',
       '여행동기1순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식',
       '여행동기1순위_여행 동반자와의 친밀감 및 유대감 증진', '여행동기1순위_역사 탐방, 문화적 경험 등 교육적 동기',
       '여행동기1순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피',
       '여행동기1순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기', '여행동기2순위_새로운 경험 추구',
       '여행동기2순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식',
       '여행동기2순위_여행 동반자와의 친밀감 및 유대감 증진', '여행동기2순위_역사 탐방, 문화적 경험 등 교육적 동기',
       '여행동기2순위_운동, 건강 증진 및 충전', '여행동기2순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피',
       '여행동기2순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기', '여행동기3순위_SNS 사진 등록 등 과시',
       '여행동기3순위_새로운 경험 추구', '여행동기3순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식',
       '여행동기3순위_여행 동반자와의 친밀감 및 유대감 증진', '여행동기3순위_역사 탐방, 문화적 경험 등 교육적 동기',
       '여행동기3순위_운동, 건강 증진 및 충전', '여행동기3순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피',
       '여행동기3순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기', '자연/도시_전자 매우선호',
       '자연/도시_전자 조금선호', '자연/도시_전자 중간선호', '자연/도시_후자 매우선호', '자연/도시_후자 조금선호',
       '자연/도시_후자 중간선호', '숙박/당일_전자 매우선호', '숙박/당일_전자 조금선호', '숙박/당일_전자 중간선호',
       '숙박/당일_후자 매우선호', '숙박/당일_후자 조금선호', '숙박/당일_후자 중간선호', '익숙한/새로운_전자 매우선호',
       '익숙한/새로운_전자 조금선호', '익숙한/새로운_전자 중간선호', '익숙한/새로운_후자 매우선호',
       '익숙한/새로운_후자 조금선호', '익숙한/새로운_후자 중간선호', '편하지만비싼숙소/불편하지만저렴한숙소_전자 매우선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_전자 조금선호', '편하지만비싼숙소/불편하지만저렴한숙소_전자 중간선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_후자 매우선호', '편하지만비싼숙소/불편하지만저렴한숙소_후자 조금선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_후자 중간선호', '휴양휴식/체험활동_전자 매우선호', '휴양휴식/체험활동_전자 조금선호',
       '휴양휴식/체험활동_전자 중간선호', '휴양휴식/체험활동_후자 매우선호', '휴양휴식/체험활동_후자 조금선호',
       '휴양휴식/체험활동_후자 중간선호', '잘알려지지않은/알려진_전자 매우선호', '잘알려지지않은/알려진_전자 조금선호',
       '잘알려지지않은/알려진_전자 중간선호', '잘알려지지않은/알려진_후자 매우선호', '잘알려지지않은/알려진_후자 조금선호',
       '잘알려지지않은/알려진_후자 중간선호', '계획에따른/상황에따른_전자 매우선호', '계획에따른/상황에따른_전자 조금선호',
       '계획에따른/상황에따른_전자 중간선호', '계획에따른/상황에따른_후자 매우선호', '계획에따른/상황에따른_후자 조금선호',
       '계획에따른/상황에따른_후자 중간선호', '사진촬영중요하지않음/사진촬영중요_전자 매우선호',
       '사진촬영중요하지않음/사진촬영중요_전자 조금선호', '사진촬영중요하지않음/사진촬영중요_전자 중간선호',
       '사진촬영중요하지않음/사진촬영중요_후자 매우선호', '사진촬영중요하지않음/사진촬영중요_후자 조금선호',
       '사진촬영중요하지않음/사진촬영중요_후자 중간선호',
       'EI', 'SN', 'FT', 'JP']]
mbti_motivation.head()

,MBTI,여행동기1순위_새로운 경험 추구,"여행동기1순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행동기1순위_여행 동반자와의 친밀감 및 유대감 증진,"여행동기1순위_역사 탐방, 문화적 경험 등 교육적 동기","여행동기1순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행동기1순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,여행동기2순위_새로운 경험 추구,"여행동기2순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행동기2순위_여행 동반자와의 친밀감 및 유대감 증진,...,사진촬영중요하지않음/사진촬영중요_전자 매우선호,사진촬영중요하지않음/사진촬영중요_전자 조금선호,사진촬영중요하지않음/사진촬영중요_전자 중간선호,사진촬영중요하지않음/사진촬영중요_후자 매우선호,사진촬영중요하지않음/사진촬영중요_후자 조금선호,사진촬영중요하지않음/사진촬영중요_후자 중간선호,EI,SN,FT,JP
0,ENFP,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,E,N,F,P
1,INFP,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,I,N,F,P
2,INTJ,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,I,N,T,J
3,ENFP,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,E,N,F,P
4,ENTP,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,E,N,T,P


In [ ]:
# 성별 O, 동기 O, 라벨링
mbti_all = research_label[['MBTI', '성별', '여행동기1순위', '여행동기2순위', '여행동기3순위', '자연/도시', '숙박/당일', '익숙한/새로운', '편하지만비싼숙소/불편하지만저렴한숙소',
                     '휴양휴식/체험활동', '잘알려지지않은/알려진', '계획에따른/상황에따른', '사진촬영중요하지않음/사진촬영중요',
                     'EI', 'SN', 'FT', 'JP']]
mbti_all.head()

,MBTI,성별,여행동기1순위,여행동기2순위,여행동기3순위,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,계획에따른/상황에따른,사진촬영중요하지않음/사진촬영중요,EI,SN,FT,JP
0,ENFP,1,1,7,3,5,3,5,2,6,5,3,7,E,N,F,P
1,INFP,1,3,4,7,2,3,7,2,3,1,2,2,I,N,F,P
2,INTJ,1,2,1,7,2,2,6,3,6,5,6,6,I,N,T,J
3,ENFP,1,1,7,3,5,3,5,2,6,5,3,7,E,N,F,P
4,ENTP,0,1,2,7,2,2,6,2,2,6,6,5,E,N,T,P


### AI HUB

In [ ]:
# 성별 X, 동기 X, 원핫인코딩
mbti_traveler = traveler_oh[['자연/도시_전자 매우선호',
       '자연/도시_전자 조금선호', '자연/도시_전자 중간선호', '자연/도시_후자 매우선호', '자연/도시_후자 조금선호',
       '자연/도시_후자 중간선호', '숙박/당일_전자 매우선호', '숙박/당일_전자 조금선호', '숙박/당일_전자 중간선호',
       '숙박/당일_후자 매우선호', '숙박/당일_후자 조금선호', '숙박/당일_후자 중간선호', '익숙한/새로운_전자 매우선호',
       '익숙한/새로운_전자 조금선호', '익숙한/새로운_전자 중간선호', '익숙한/새로운_후자 매우선호',
       '익숙한/새로운_후자 조금선호', '익숙한/새로운_후자 중간선호', '편하지만비싼숙소/불편하지만저렴한숙소_전자 매우선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_전자 조금선호', '편하지만비싼숙소/불편하지만저렴한숙소_전자 중간선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_후자 매우선호', '편하지만비싼숙소/불편하지만저렴한숙소_후자 조금선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_후자 중간선호', '휴양휴식/체험활동_전자 매우선호', '휴양휴식/체험활동_전자 조금선호',
       '휴양휴식/체험활동_전자 중간선호', '휴양휴식/체험활동_후자 매우선호', '휴양휴식/체험활동_후자 조금선호',
       '휴양휴식/체험활동_후자 중간선호', '잘알려지지않은/알려진_전자 매우선호', '잘알려지지않은/알려진_전자 조금선호',
       '잘알려지지않은/알려진_전자 중간선호', '잘알려지지않은/알려진_후자 매우선호', '잘알려지지않은/알려진_후자 조금선호',
       '잘알려지지않은/알려진_후자 중간선호', '계획에따른/상황에따른_전자 매우선호', '계획에따른/상황에따른_전자 조금선호',
       '계획에따른/상황에따른_전자 중간선호', '계획에따른/상황에따른_후자 매우선호', '계획에따른/상황에따른_후자 조금선호',
       '계획에따른/상황에따른_후자 중간선호', '사진촬영중요하지않음/사진촬영중요_전자 매우선호',
       '사진촬영중요하지않음/사진촬영중요_전자 조금선호', '사진촬영중요하지않음/사진촬영중요_전자 중간선호',
       '사진촬영중요하지않음/사진촬영중요_후자 매우선호', '사진촬영중요하지않음/사진촬영중요_후자 조금선호',
       '사진촬영중요하지않음/사진촬영중요_후자 중간선호']]
mbti_traveler.head()

,자연/도시_전자 매우선호,자연/도시_전자 조금선호,자연/도시_전자 중간선호,자연/도시_후자 매우선호,자연/도시_후자 조금선호,자연/도시_후자 중간선호,숙박/당일_전자 매우선호,숙박/당일_전자 조금선호,숙박/당일_전자 중간선호,숙박/당일_후자 매우선호,...,계획에따른/상황에따른_전자 중간선호,계획에따른/상황에따른_후자 매우선호,계획에따른/상황에따른_후자 조금선호,계획에따른/상황에따른_후자 중간선호,사진촬영중요하지않음/사진촬영중요_전자 매우선호,사진촬영중요하지않음/사진촬영중요_전자 조금선호,사진촬영중요하지않음/사진촬영중요_전자 중간선호,사진촬영중요하지않음/사진촬영중요_후자 매우선호,사진촬영중요하지않음/사진촬영중요_후자 조금선호,사진촬영중요하지않음/사진촬영중요_후자 중간선호
0,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,0,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,0
4,0,0,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0


In [ ]:
# 성별 O, 동기 X, 라벨링
mbti_gender_traveler = traveler_label[['성별', '자연/도시', '숙박/당일', '익숙한/새로운', '편하지만비싼숙소/불편하지만저렴한숙소',
                        '휴양휴식/체험활동', '잘알려지지않은/알려진', '계획에따른/상황에따른', '사진촬영중요하지않음/사진촬영중요']]
mbti_gender_traveler.head()

,성별,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,계획에따른/상황에따른,사진촬영중요하지않음/사진촬영중요
0,1,1,1,7,5,1,1,5,2
1,1,2,1,1,5,3,3,2,6
2,0,1,3,2,3,2,2,3,2
3,0,2,7,2,6,6,6,6,2
4,0,7,7,1,1,3,1,7,7


In [ ]:
# 성별 X, 동기 O, 원핫인코딩
mbti_motivation_traveler = traveler_oh[['여행동기1순위_새로운 경험 추구',
       '여행동기1순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식',
       '여행동기1순위_여행 동반자와의 친밀감 및 유대감 증진', '여행동기1순위_역사 탐방, 문화적 경험 등 교육적 동기',
       '여행동기1순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피',
       '여행동기1순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기', '여행동기2순위_새로운 경험 추구',
       '여행동기2순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식',
       '여행동기2순위_여행 동반자와의 친밀감 및 유대감 증진', '여행동기2순위_역사 탐방, 문화적 경험 등 교육적 동기',
       '여행동기2순위_운동, 건강 증진 및 충전', '여행동기2순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피',
       '여행동기2순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기', '여행동기3순위_SNS 사진 등록 등 과시',
       '여행동기3순위_새로운 경험 추구', '여행동기3순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식',
       '여행동기3순위_여행 동반자와의 친밀감 및 유대감 증진', '여행동기3순위_역사 탐방, 문화적 경험 등 교육적 동기',
       '여행동기3순위_운동, 건강 증진 및 충전', '여행동기3순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피',
       '여행동기3순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기', '자연/도시_전자 매우선호',
       '자연/도시_전자 조금선호', '자연/도시_전자 중간선호', '자연/도시_후자 매우선호', '자연/도시_후자 조금선호',
       '자연/도시_후자 중간선호', '숙박/당일_전자 매우선호', '숙박/당일_전자 조금선호', '숙박/당일_전자 중간선호',
       '숙박/당일_후자 매우선호', '숙박/당일_후자 조금선호', '숙박/당일_후자 중간선호', '익숙한/새로운_전자 매우선호',
       '익숙한/새로운_전자 조금선호', '익숙한/새로운_전자 중간선호', '익숙한/새로운_후자 매우선호',
       '익숙한/새로운_후자 조금선호', '익숙한/새로운_후자 중간선호', '편하지만비싼숙소/불편하지만저렴한숙소_전자 매우선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_전자 조금선호', '편하지만비싼숙소/불편하지만저렴한숙소_전자 중간선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_후자 매우선호', '편하지만비싼숙소/불편하지만저렴한숙소_후자 조금선호',
       '편하지만비싼숙소/불편하지만저렴한숙소_후자 중간선호', '휴양휴식/체험활동_전자 매우선호', '휴양휴식/체험활동_전자 조금선호',
       '휴양휴식/체험활동_전자 중간선호', '휴양휴식/체험활동_후자 매우선호', '휴양휴식/체험활동_후자 조금선호',
       '휴양휴식/체험활동_후자 중간선호', '잘알려지지않은/알려진_전자 매우선호', '잘알려지지않은/알려진_전자 조금선호',
       '잘알려지지않은/알려진_전자 중간선호', '잘알려지지않은/알려진_후자 매우선호', '잘알려지지않은/알려진_후자 조금선호',
       '잘알려지지않은/알려진_후자 중간선호', '계획에따른/상황에따른_전자 매우선호', '계획에따른/상황에따른_전자 조금선호',
       '계획에따른/상황에따른_전자 중간선호', '계획에따른/상황에따른_후자 매우선호', '계획에따른/상황에따른_후자 조금선호',
       '계획에따른/상황에따른_후자 중간선호', '사진촬영중요하지않음/사진촬영중요_전자 매우선호',
       '사진촬영중요하지않음/사진촬영중요_전자 조금선호', '사진촬영중요하지않음/사진촬영중요_전자 중간선호',
       '사진촬영중요하지않음/사진촬영중요_후자 매우선호', '사진촬영중요하지않음/사진촬영중요_후자 조금선호',
       '사진촬영중요하지않음/사진촬영중요_후자 중간선호']]
mbti_motivation_traveler.head()

,여행동기1순위_새로운 경험 추구,"여행동기1순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행동기1순위_여행 동반자와의 친밀감 및 유대감 증진,"여행동기1순위_역사 탐방, 문화적 경험 등 교육적 동기","여행동기1순위_일상적인 환경 및 역할에서의 탈출, 지루함 탈피",여행동기1순위_진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,여행동기2순위_새로운 경험 추구,"여행동기2순위_쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",여행동기2순위_여행 동반자와의 친밀감 및 유대감 증진,"여행동기2순위_역사 탐방, 문화적 경험 등 교육적 동기",...,계획에따른/상황에따른_전자 중간선호,계획에따른/상황에따른_후자 매우선호,계획에따른/상황에따른_후자 조금선호,계획에따른/상황에따른_후자 중간선호,사진촬영중요하지않음/사진촬영중요_전자 매우선호,사진촬영중요하지않음/사진촬영중요_전자 조금선호,사진촬영중요하지않음/사진촬영중요_전자 중간선호,사진촬영중요하지않음/사진촬영중요_후자 매우선호,사진촬영중요하지않음/사진촬영중요_후자 조금선호,사진촬영중요하지않음/사진촬영중요_후자 중간선호
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,0,0,0,0,0,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0


In [ ]:
# 성별 O, 동기 O, 라벨링
mbti_all_traveler = traveler_label[['성별', '여행동기1순위', '여행동기2순위', '여행동기3순위', '자연/도시', '숙박/당일', '익숙한/새로운', '편하지만비싼숙소/불편하지만저렴한숙소',
                     '휴양휴식/체험활동', '잘알려지지않은/알려진', '계획에따른/상황에따른', '사진촬영중요하지않음/사진촬영중요']]
mbti_all_traveler.head()

,성별,여행동기1순위,여행동기2순위,여행동기3순위,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,계획에따른/상황에따른,사진촬영중요하지않음/사진촬영중요
0,1,10.0,NaN,NaN,1,1,7,5,1,1,5,2
1,1,4.0,7.0,1.0,2,1,1,5,3,3,2,6
2,0,8.0,1.0,6.0,1,3,2,3,2,2,3,2
3,0,3.0,1.0,2.0,2,7,2,6,6,6,6,2
4,0,1.0,2.0,7.0,7,7,1,1,3,1,7,7


# 데이터셋 분리

In [ ]:
# 성별 X, 동기 O, 원핫, EI 분리
X_EI_motivation = mbti_motivation.iloc[:, 1:-4]
y_EI_motivation = mbti_motivation.iloc[:, -4]
X_train_EI_motivation, X_test_EI_motivation, y_train_EI_motivation, y_test_EI_motivation = train_test_split(X_EI_motivation, y_EI_motivation, test_size=0.2, random_state=0)

In [ ]:
# 성별 O, 동기 O, 라벨링, SN 분리
X_SN_all = mbti_all.iloc[:, 1:-4]
y_SN_all = mbti_all.iloc[:, -3]
X_train_SN_all, X_test_SN_all, y_train_SN_all, y_test_SN_all = train_test_split(X_SN_all, y_SN_all, test_size=0.2, random_state=0)

In [ ]:
# 성별 X, 동기 X, 원핫, FT 분리
X_FT = mbti.iloc[:, 1:-4]
y_FT = mbti.iloc[:, -2]
X_train_FT, X_test_FT, y_train_FT, y_test_FT = train_test_split(X_FT, y_FT, test_size=0.2, random_state=0)

In [ ]:
# 성별 O, 동기 X, 라벨링, JP 분리
X_JP_gender = mbti_gender.iloc[:, 1:-4]
y_JP_gender = mbti_gender.iloc[:, -1]
X_train_JP_gender, X_test_JP_gender, y_train_JP_gender, y_test_JP_gender = train_test_split(X_JP_gender, y_JP_gender, test_size=0.2, random_state=0)

# MBTI 예측모델

In [ ]:
# E/I, 성별 X, 동기 O, 원핫, DT
dt_clf_EI_motivation = DecisionTreeClassifier(random_state=0)
dt_clf_EI_motivation.fit(X_train_EI_motivation, y_train_EI_motivation)
pred_EI_motivation = dt_clf_EI_motivation.predict(X_test_EI_motivation)

In [ ]:
# S/N, 성별 O, 동기 O, 라벨링, CB
cb_clf_SN_all = CatBoostClassifier(random_state=0)
cb_clf_SN_all.fit(X_train_SN_all, y_train_SN_all)
pred_SN_all = cb_clf_SN_all.predict(X_test_SN_all)

Learning rate set to 0.004045
0:	learn: 0.6917174	total: 50ms	remaining: 50s
1:	learn: 0.6907943	total: 51.9ms	remaining: 25.9s
2:	learn: 0.6891392	total: 54.9ms	remaining: 18.2s
3:	learn: 0.6881452	total: 57.8ms	remaining: 14.4s
4:	learn: 0.6863636	total: 61.4ms	remaining: 12.2s
5:	learn: 0.6846937	total: 63.9ms	remaining: 10.6s
6:	learn: 0.6834876	total: 66.8ms	remaining: 9.47s
7:	learn: 0.6823330	total: 70.3ms	remaining: 8.71s
8:	learn: 0.6809940	total: 72.3ms	remaining: 7.96s
9:	learn: 0.6797890	total: 76ms	remaining: 7.52s
10:	learn: 0.6786463	total: 88.2ms	remaining: 7.93s
11:	learn: 0.6776081	total: 91.3ms	remaining: 7.52s
12:	learn: 0.6762090	total: 94.5ms	remaining: 7.18s
13:	learn: 0.6748837	total: 97.8ms	remaining: 6.89s
14:	learn: 0.6734156	total: 101ms	remaining: 6.63s
15:	learn: 0.6727406	total: 104ms	remaining: 6.41s
16:	learn: 0.6716849	total: 107ms	remaining: 6.21s
17:	learn: 0.6705493	total: 111ms	remaining: 6.03s
18:	learn: 0.6693117	total: 114ms	remaining: 5.87s
19:

In [ ]:
# F/T, 성별 X, 동기 X, 원핫, RLM
rf_clf_FT = RandomForestClassifier(random_state=0)
rf_clf_FT.fit(X_train_FT, y_train_FT)
pred_FT = rf_clf_FT.predict(X_test_FT)

In [ ]:
# J/P, 성별 O, 동기 X, 라벨링, CB
cb_clf_JP_gender = CatBoostClassifier(random_state=0)
cb_clf_JP_gender.fit(X_train_JP_gender, y_train_JP_gender)
pred_JP_gender = cb_clf_JP_gender.predict(X_test_JP_gender)

Learning rate set to 0.004045
0:	learn: 0.6909228	total: 2.42ms	remaining: 2.41s
1:	learn: 0.6888509	total: 4.13ms	remaining: 2.06s
2:	learn: 0.6872187	total: 5.49ms	remaining: 1.82s
3:	learn: 0.6856616	total: 6.85ms	remaining: 1.7s
4:	learn: 0.6833263	total: 8.18ms	remaining: 1.63s
5:	learn: 0.6822867	total: 9.3ms	remaining: 1.54s
6:	learn: 0.6802652	total: 10.4ms	remaining: 1.47s
7:	learn: 0.6785175	total: 18ms	remaining: 2.23s
8:	learn: 0.6768731	total: 20.3ms	remaining: 2.23s
9:	learn: 0.6750683	total: 22.1ms	remaining: 2.18s
10:	learn: 0.6727651	total: 23.6ms	remaining: 2.12s
11:	learn: 0.6709386	total: 25ms	remaining: 2.06s
12:	learn: 0.6688980	total: 28.4ms	remaining: 2.16s
13:	learn: 0.6667723	total: 34.7ms	remaining: 2.44s
14:	learn: 0.6645609	total: 36.2ms	remaining: 2.38s
15:	learn: 0.6636362	total: 37.8ms	remaining: 2.32s
16:	learn: 0.6621697	total: 39.1ms	remaining: 2.26s
17:	learn: 0.6600701	total: 40.9ms	remaining: 2.23s
18:	learn: 0.6582925	total: 42.6ms	remaining: 2.2s

# AI HUB MBTI 예측

In [ ]:
# E/I
pred_EI_traveler = dt_clf_EI_motivation.predict(mbti_motivation_traveler)

# S/N
pred_SN_traveler = cb_clf_SN_all.predict(mbti_all_traveler)

# F/T
pred_FT_traveler = rf_clf_FT.predict(mbti_traveler)

# J/P
pred_JP_traveler = cb_clf_JP_gender.predict(mbti_gender_traveler)

In [ ]:
traveler_rev['EI'] = pred_EI_traveler
traveler_rev['SN'] = pred_SN_traveler
traveler_rev['FT'] = pred_FT_traveler
traveler_rev['JP'] = pred_JP_traveler
traveler_rev

,성별,자연/도시,숙박/당일,익숙한/새로운,편하지만비싼숙소/불편하지만저렴한숙소,휴양휴식/체험활동,잘알려지지않은/알려진,계획에따른/상황에따른,사진촬영중요하지않음/사진촬영중요,여행동기1순위,여행동기2순위,여행동기3순위,EI,SN,FT,JP
0,여,1,1,7,5,1,1,5,2,10.0,NaN,NaN,I,N,F,P
1,여,2,1,1,5,3,3,2,6,4.0,7.0,1.0,E,S,T,J
2,남,1,3,2,3,2,2,3,2,8.0,1.0,6.0,E,N,F,J
3,남,2,7,2,6,6,6,6,2,3.0,1.0,2.0,E,N,F,P
4,남,7,7,1,1,3,1,7,7,1.0,2.0,7.0,E,N,F,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3799,남,5,1,1,3,2,3,3,3,1.0,3.0,2.0,I,S,F,J
3800,여,6,2,1,6,6,6,5,2,1.0,2.0,3.0,E,S,F,P
3801,여,3,2,3,3,3,5,5,5,1.0,7.0,3.0,E,S,F,P
3802,여,2,2,3,3,2,3,2,6,2.0,1.0,5.0,I,S,T,J


In [ ]:
print(traveler_rev['EI'].value_counts())
print(traveler_rev['SN'].value_counts())
print(traveler_rev['FT'].value_counts())
print(traveler_rev['JP'].value_counts())

I    2566
E    1238
Name: EI, dtype: int64
N    2011
S    1793
Name: SN, dtype: int64
F    2978
T     826
Name: FT, dtype: int64
P    2083
J    1721
Name: JP, dtype: int64


In [ ]:
traveler['MBTI'] = traveler_rev['EI'] + traveler_rev['SN'] + traveler_rev['FT'] + traveler_rev['JP']
traveler

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,...,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM,MBTI
0,a006685,41,여,30,5,1.0,2.0,2,3.0,NaN,...,2,경기도,인천,2인 가족 여행,2022-10-09~2022-10-09,10.0,NaN,NaN,1,INFP
1,a014274,11,여,20,6,1.0,1.0,4,2.0,NaN,...,6,서울특별시,경기,나홀로 여행,2022-11-01~2022-11-03,4.0,7.0,1.0,0,ESTJ
2,a001158,41,남,30,6,1.0,1.0,1,3.0,NaN,...,2,경기도,경기,나홀로 여행,2022-09-01~2022-09-03,8.0,1.0,6.0,0,ENFJ
3,a005594,28,남,30,6,1.0,2.0,4,3.0,NaN,...,2,인천광역시,경기,자녀 동반 여행,2022-10-08~2022-10-08,3.0,1.0,2.0,3,ENFP
4,a007033,11,남,20,4,1.0,1.0,3,12.0,NaN,...,7,서울특별시,경기,나홀로 여행,2022-10-15~2022-10-15,1.0,2.0,7.0,0,ENFP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3799,d000922,41,남,50,6,1.0,1.0,2,3.0,NaN,...,3,경기도,제주,부모 동반 여행,2022-08-23~2022-08-26,1.0,3.0,2.0,4,ISFJ
3800,d010626,50,여,40,5,1.0,2.0,4,13.0,NaN,...,2,제주특별자치도,제주,자녀 동반 여행,2022-11-05~2022-11-06,1.0,2.0,3.0,3,ESFP
3801,d001143,11,여,30,7,1.0,2.0,4,2.0,NaN,...,5,서울특별시,제주,자녀 동반 여행,2022-09-17~2022-09-21,1.0,7.0,3.0,3,ESFP
3802,b004853,48,여,30,7,1.0,1.0,3,3.0,NaN,...,6,경상남도,부산,나홀로 여행,2022-09-30~2022-10-01,2.0,1.0,5.0,0,ISTJ


# CSV 출력

In [ ]:
traveler.to_csv("/content/drive/MyDrive/3조(Rodi알려조)/9. 데이터셋/002.권역별데이터(MBTI예측)/여행객(MBTI예측).csv", index=False)